### Coding Exercises

Consider the following classes:

In [71]:


import inspect

class AutoRepr:
    @classmethod
    def vars_signature(cls, obj):
        """return a map of attribute names to __init__ arguments names"""
        params = inspect.signature(cls.__init__).parameters.keys()
        inits = {}
        # print('inits: vars(cls)=', vars(obj))
        for attr in vars(obj):
            par = next((
                par for par in params
                if attr in par
            ), None)
            # print(f'inits: attr={attr}, par={par}')
            if par is not None:
                inits[attr] = par
        # print('inits: inits=', inits)
        return inits

    @classmethod
    def signature_vars(cls, obj):
        """return a map of __init__ arguments names to attribute names"""
        return {
            val: key for key, val in
            cls.vars_signature().items()
        }

    @property
    def init_kwargs(self):
        return {
            key: getattr(self, attr, None)
            for key, attr in self.signature_vars().items()
        }
    
    def __repr__(self):
        args = ', '.join(
            f"{key}='{str(val)}'"
            for key, val in self.init_kwargs.items()
        )
        return f"{self.__class__.__name__}({args})"

class Stock(AutoRepr):
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
class Trade(AutoRepr):
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume
        
stock1 = Stock('TSLA', date(2018, 11, 22), 
  Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607)

print(stock1)

inits: vars(cls)= {'__module__': '__main__', '__init__': <function Stock.__init__ at 0x7ff2c0019160>, '__doc__': None}
inits: attr=__module__, par=None
inits: attr=__init__, par=None
inits: attr=__doc__, par=None
inits: inits= {}
Stock()


#### Exercise 1

Given the above class, write a custom `JSONEncoder` class to **serialize** dictionaries that contain instances of these particular classes. Keep in mind that you will want to deserialize the data too - so you will need some technique to indicate the object type in your serialization.

For example you may have an object such as this one that needs to be serialized:

In [2]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [9]:
stock1 = Stock('TSLA', date(2018, 11, 22), 
  Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607)

dec1 = Decimal('338.19')
date.today().isoformat()
datetime.now().isoformat()

print(vars(stock1))

{'symbol': 'TSLA', 'date': datetime.date(2018, 11, 22), 'open': Decimal('338.19'), 'high': Decimal('338.64'), 'low': Decimal('337.60'), 'close': Decimal('338.19'), 'volume': 365607}


In [37]:

import json
import functools

@functools.singledispatch
def custom_dumps(arg)->str:
    return str(arg)

@custom_dumps.register(Decimal)
def _(arg:Decimal)->str:
    return f"Decimal({str(arg)})"

@custom_dumps.register(date)
def _(arg:date)->str:
    return {'class_date': arg.isoformat()}

@custom_dumps.register(datetime)
def _(arg:datetime)->str:
    return {'class_datetime': arg.isoformat()}

@custom_dumps.register(Stock)
def _(arg:Stock)->str:
    return {'class_stock': vars(arg)}

@custom_dumps.register(Trade)
def _(arg:Trade)->str:
    return {'class_trade': vars(arg)}

class CustomJSONEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **{**kwargs, 'indent': 2})
    
    def default(self, arg):
        return custom_dumps(arg)

result1 = json.dumps(activity, default=custom_dumps, indent=2)
result2 = json.dumps(activity, cls=CustomJSONEncoder)

print(result1 == result2)
with open('activity_1.json', 'w', newline='') as file:
    file.write(result1)
with open('activity_2.json', 'w', newline='') as file:
    file.write(result2)

True


Hint: You can modify the classes if you need to.

#### Exercise 2

Write code to reverse the serialization you just created. Write a custom decoder that can deserialize a JSON structure containing `Stock` and `Trade` objects. 

In [42]:

import re

decimal_re = re.compile('Decimal\(.+\)')

print(decimal_re.search('Decimal(2.456)'))
print(decimal_re.search('2.456'))

arg = 'Decimal(2.456)'
if isinstance(arg, str) and decimal_re.match(arg):
    val = Decimal(arg.strip('Decimal()'))
    print(type(val))

<re.Match object; span=(0, 14), match='Decimal(2.456)'>
None
<class 'decimal.Decimal'>


In [65]:


def my_hook(arg):
    # print('\n\nreading:', type(arg), arg)
    for key, val in arg.items():
        if decimal_re.match(val):
            arg.update({key: Decimal(val.strip(' Decimal()'))})
    for label, Class in (
            ('class_date', date),
            ('class_datetime', datetime)
        ):
        return Class.fromisoformat(arg[label])
    for label, Class in (
            ('class_stock', Stock),
            ('class_trade', Trade)
        ):
        kwargs = arg[label]
        init_map = Class.vars_signature()
        return Class(**{init_map[k]:v for k,v in kwargs.items()})
    else:
        return arg


# activity_loaded = json.loads(result1, object_pair_hook=my_pair_hook)
activity_loaded = json.loads(result1, object_hook=my_hook)
print(activity_loaded)

TypeError: expected string or bytes-like object

#### Exercise 3

Do the same serialization and deserialization, but using `Marshmallow`.